In [1]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from os.path import join as p_join

import seaborn as sns


##################################
## GLOBAL SETTINGS ###############
##################################
plt.rcParams["figure.figsize"] = (12,8)

In [2]:
###############################
##### IMPORT DL Depensies #####
###############################
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset, DataLoader

from tqdm import tqdm


def seed_all(seed=42):

    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print("[ Using Seed : ", seed, " ]")

####################################
#####   SEED ALL EXPERIMENTS   #####
####################################
seed_all()   

[ Using Seed :  42  ]


In [3]:
import sys
sys.path.append("../..") # Adds higher directory to python modules path.
from utils import utils

### Handle Function

In [4]:
#################################
#### MAKE TEST TRAIN SPLIT   ####
#################################
def create_dataloaders(CONFIG, X, Y):
    from sklearn.model_selection import train_test_split

    X_train, X_test, Y_train, Y_test = train_test_split(
        X, Y, test_size=0.2, random_state=42)


    train_dataset = TensorDataset(torch.Tensor(X_train), torch.Tensor(Y_train))
    test_dataset = TensorDataset(torch.Tensor(X_test), torch.Tensor(Y_test))

    train_dataloader = DataLoader(train_dataset,
                                  shuffle=True,
                                  num_workers=CONFIG['cpu_workers'],
                                  batch_size=CONFIG['train_batch_size'],
                                  drop_last=True)

    test_dataloader = DataLoader(test_dataset,
                                 shuffle=False,
                                 num_workers=CONFIG['cpu_workers'],
                                 batch_size=CONFIG['test_batch_size'],
                                 drop_last=False)

    return train_dataloader, test_dataloader

### Try extra featrues

In [5]:
from utils import dl_models

def create_dataset(folders, verbose=False, feature_nums=16):
    X_merged, Y_merged = [], []
    for folder_path in tqdm(folders):
        X_path = p_join(folder_path, '2nd_exp_Input.txt')
        Y_path_down = p_join(folder_path, '2nd_exp_Topology_down.txt')
        Y_path_up = p_join(folder_path, '2nd_exp_Topology_up.txt')
        params_path = p_join(folder_path, '2nd_exp_Parameters.txt')

        X = utils.select_n_center_features(pd.read_csv(X_path).values, feature_nums)
        X_params = pd.read_csv(params_path).values[:,:-5] # remove last values
        Y_down = pd.read_csv(Y_path_down).values
        Y_up = pd.read_csv(Y_path_up).values
        Y = np.array([utils.map_classes(Y_up[i], Y_down[i]) for i in range(Y_up.shape[0])])
        X = np.concatenate([X, X_params], axis=1)

        X_merged.append(X)
        Y_merged.append(Y)

    # Merge and shuffle it!
    X, Y = np.concatenate(X_merged, axis=0), np.concatenate(Y_merged, axis=0)
    indexes_for_shuffle = np.random.permutation(np.arange(X.shape[0]))
    X = X[indexes_for_shuffle]
    Y = Y[indexes_for_shuffle]
    if verbose:
        print('Dataset cteated!')
    return X, Y

In [6]:
data_path = p_join('..', '..', 'data_2')
FOLDERS = [p_join(os.path.abspath(data_path), item) for item in os.listdir(data_path) if 'L=10.6' in item]

X, Y = create_dataset(FOLDERS)
X = utils.normalize_data(X)

100%|███████████████████████████████████████████| 16/16 [00:01<00:00,  8.91it/s]


In [7]:
CONFIG = {'ephs': 5,
          'train_batch_size': 1024,
          'test_batch_size': 64,
          'cpu_workers': 2,
          'save_ckpts': False}
net = dl_models.MlpNeLight(X.shape[1])
utils.count_params(net)
train_dataloader, test_dataloader = create_dataloaders(CONFIG, X, Y)
optimizer = torch.optim.Adam(net.parameters(), lr=CONFIG.get('lr', 5e-3))
utils.run_training(net, optimizer, CONFIG, train_dataloader, test_dataloader)

[INFO]: Model "MlpNeLight" has 162456 trainable parameters


100%|███████████████████████████████████████████| 49/49 [00:19<00:00,  2.46it/s]


Epoch: 1/5, 	 total train loss: 0.5388973276225888


100%|█████████████████████████████████████████| 200/200 [00:12<00:00, 16.39it/s]


Epoch: 1/5, 	 total score test: 77.87762756896147, [best score: 77.87762756896147]



...

Epoch: 4/5, 	 total score test: 94.02985074626865, [best score: 94.02985074626865]



100%|███████████████████████████████████████████| 49/49 [00:06<00:00,  7.37it/s]


Epoch: 5/5, 	 total train loss: 0.1407565714753404


100%|████████████████████████████████████████| 200/200 [00:01<00:00, 117.57it/s]

Epoch: 5/5, 	 total score test: 94.56903961866063, [best score: 94.56903961866063]



### Try Ml regression

In [8]:
def reg_create_dataset(folders, verbose=False, feature_nums=16):
    X_merged, Y_merged_1, Y_merged_2 = [], [], []
    for folder_path in tqdm(folders):
        X_path = p_join(folder_path, '2nd_exp_Input.txt')
        Y_path_down = p_join(folder_path, '2nd_exp_Topology_down.txt')
        Y_path_up = p_join(folder_path, '2nd_exp_Topology_up.txt')
        params_path = p_join(folder_path, '2nd_exp_Parameters.txt')

        X = utils.select_n_center_features(pd.read_csv(X_path).values, feature_nums)
        Y1 = pd.read_csv(params_path).values[:, 0]
        Y_down = pd.read_csv(Y_path_down).values
        Y_up = pd.read_csv(Y_path_up).values
        Y2 = np.array([utils.map_classes(Y_up[i], Y_down[i]) for i in range(Y_up.shape[0])])

        X_merged.append(X)
        Y_merged_1.append(Y1)
        Y_merged_2.append(Y2)

    # Merge and shuffle it!
    X, Y1, Y2 = np.concatenate(X_merged, axis=0), np.concatenate(Y_merged_1, axis=0), np.concatenate(Y_merged_2, axis=0)
    indexes_for_shuffle = np.random.permutation(np.arange(X.shape[0]))
    X = X[indexes_for_shuffle]
    Y1 = Y1[indexes_for_shuffle]
    Y2 = Y2[indexes_for_shuffle]
    if verbose:
        print('Dataset cteated!')
    return X, Y1, Y2

In [9]:
FOLDERS = [p_join(os.path.abspath(data_path), item) for item in os.listdir(data_path) if 'L=10.6' in item]
X, EXT_FEAT, CLASS = reg_create_dataset(FOLDERS)
X = utils.normalize_data(X)
EXT_FEAT = utils.normalize_data(EXT_FEAT)

100%|███████████████████████████████████████████| 16/16 [00:01<00:00, 10.16it/s]


In [10]:
### Split it!
n = X.shape[0] // 2
X1, X2 = X[:n], X[n:]
EXT_FEAT1, EXT_FEAT2 = EXT_FEAT[:n], EXT_FEAT[n:]
CLASS1, CLASS2 = CLASS[:n], CLASS[n:]

In [11]:
from sklearn.linear_model import LinearRegression

ML_CONFIG = {'scoring': 'max_error',
             'cv': 5,
             'n_jobs': None}

In [12]:
model = LinearRegression()
utils.calc_ml_method(model, ML_CONFIG, X1, EXT_FEAT1)

{'max_error': {'fit_time': array([0.01751328, 0.03773999, 0.04158902, 0.02996969, 0.03684521]),
  'score_time': array([0.01316237, 0.00502348, 0.0143249 , 0.00235367, 0.00156212]),
  'test_score': array([-2.2723536 , -2.5085041 , -2.31946085, -2.32599584, -2.34098657])}}

In [13]:
model = LinearRegression()
model.fit(X1, EXT_FEAT1)
pass

In [14]:
EXT_FEAT2_HAT = model.predict(X2)

In [15]:
XX = np.concatenate([X2, EXT_FEAT2_HAT[:, None]], axis=1)
XX.shape

(31992, 17)

In [16]:
net = dl_models.MlpNeLight(XX.shape[1])
utils.count_params(net)
train_dataloader, test_dataloader = create_dataloaders(CONFIG, XX, CLASS2)
optimizer = torch.optim.Adam(net.parameters(), lr=CONFIG.get('lr', 5e-3))
utils.run_training(net, optimizer, CONFIG, train_dataloader, test_dataloader)

[INFO]: Model "MlpNeLight" has 162456 trainable parameters


100%|███████████████████████████████████████████| 24/24 [00:02<00:00, 11.80it/s]


Epoch: 1/5, 	 total train loss: 1.1772363434235256


100%|████████████████████████████████████████| 100/100 [00:00<00:00, 183.12it/s]


Epoch: 1/5, 	 total score test: 45.538365369589, [best score: 45.538365369589]



...

Epoch: 4/5, 	 total score test: 58.58727926238475, [best score: 58.58727926238475]



100%|███████████████████████████████████████████| 24/24 [00:01<00:00, 18.75it/s]


Epoch: 5/5, 	 total train loss: 0.7672057375311852


100%|████████████████████████████████████████| 100/100 [00:00<00:00, 241.22it/s]

Epoch: 5/5, 	 total score test: 60.040631348648226, [best score: 60.040631348648226]



In [17]:
data_path = p_join('..', '..', 'data_2')
FOLDERS = [p_join(os.path.abspath(data_path), item) for item in os.listdir(data_path) if 'L=10.6' in item]

X, Y = utils.create_dataset(FOLDERS)
X = utils.normalize_data(X)

CONFIG = {'ephs': 5,
          'train_batch_size': 1024,
          'test_batch_size': 64,
          'cpu_workers': 2,
          'save_ckpts': False}
net = dl_models.MlpNeLight(X.shape[1])
utils.count_params(net)
train_dataloader, test_dataloader = create_dataloaders(CONFIG, X, Y)
optimizer = torch.optim.Adam(net.parameters(), lr=CONFIG.get('lr', 5e-3))
utils.run_training(net, optimizer, CONFIG, train_dataloader, test_dataloader)

100%|███████████████████████████████████████████| 16/16 [00:00<00:00, 17.68it/s]


[INFO]: Model "MlpNeLight" has 144068 trainable parameters


100%|███████████████████████████████████████████| 49/49 [00:02<00:00, 21.25it/s]


Epoch: 1/5, 	 total train loss: 1.1068817304105174


100%|████████████████████████████████████████| 200/200 [00:00<00:00, 270.42it/s]


...

Epoch: 4/5, 	 total score test: 62.389622567789324, [best score: 62.389622567789324]



100%|███████████████████████████████████████████| 49/49 [00:02<00:00, 21.17it/s]


Epoch: 5/5, 	 total train loss: 0.6784520404679435


100%|████████████████████████████████████████| 200/200 [00:00<00:00, 265.47it/s]

Epoch: 5/5, 	 total score test: 66.72657654137689, [best score: 66.72657654137689]



In [18]:
### Not works ((

### Try double loss

In [26]:
class Mlp(nn.Module):
    def __init__(self, in_features, reg_features):
        super().__init__()
        self.drop_p = 0.1
        self.drop = nn.Dropout(p=self.drop_p)
        self.backbone = nn.Sequential(
                                 nn.Linear(in_features, in_features * 8),
                                 nn.BatchNorm1d(in_features * 8),
                                 nn.GELU(),
                                 nn.Linear(in_features * 8, in_features * 32),
                                 nn.BatchNorm1d(in_features * 32),
                                 nn.GELU(),
                                 nn.Linear(in_features * 32, in_features * 8), 
                                 nn.BatchNorm1d(in_features * 8),
                                 nn.GELU(),
                            )
        self.lin = nn.Sequential(nn.Dropout(p=self.drop_p),
                                 nn.Linear(in_features * 8, in_features * 4),
                                 nn.BatchNorm1d(in_features * 4),
                                 nn.GELU(),
                                 nn.Dropout(p=self.drop_p),
                                 nn.Linear(in_features * 4, 4))
        self.reg = nn.Sequential(nn.Dropout(p=self.drop_p),
                                 nn.Linear(in_features * 8, in_features * 4),
                                 nn.BatchNorm1d(in_features * 4),
                                 nn.GELU(),
                                 nn.Dropout(p=self.drop_p),
                                 nn.Linear(in_features * 4, reg_features))

    def forward(self, x):
        x = self.backbone(x)
        class_ = self.lin(x)
        reg = self.reg(x)
        
        return class_, reg

In [27]:
def train_epoch(net, optimizer, dataloader, criterion, device):
    """Perform one traing epoch"""
    running_loss = 0.0
    net.train(True)
    for X, z, y in tqdm(dataloader):
        X = X.to(device)
        y = y.to(device).long()
        optimizer.zero_grad()
        out = net(X)
        loss = criterion(out, z, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    return running_loss / len(dataloader)

def test_model(net, dataloader, device):
    """Return accuracy"""
    correct, count = 0, 0
    net.eval()
    with torch.no_grad():
        for X, z, y in tqdm(dataloader):
            X = X.to(device)
            y = y.to(device).long()
            logits, _ = net(X)
            _, y_hat = torch.max(logits, dim=1)
            correct += (y_hat == y).sum().item()
            count += y_hat.shape[0]
    
    return 100 * correct / count

def run_training(net, optimizer, config, train_loader, test_loader=None):
    ### Get hyperparams from config
    epochs = config.get('ephs', 10)
    device = config.get('device', 'cpu')
    criterion = config.get('criterion', F.cross_entropy)
    ckpt_save_folder = config.get('ckpt_save_folder', './ckpts')
    save_ckpts = config.get('save_ckpts', False)
    
    ### Define local constants
    best_score = -np.inf
    
    if not os.path.exists(ckpt_save_folder):
        os.makedirs(ckpt_save_folder)
    
    ### Start trainig
    for eph in range(1, epochs + 1):
            mean_loss = train_epoch(net, optimizer, train_loader, criterion, device)
            print(f"Epoch: {eph}/{epochs}, \t total train loss: {mean_loss}")
            if test_loader:
                score = test_model(net, test_loader, device)
                if score > best_score:
                    best_score = score
                    if save_ckpts:
                        torch.save(net.state_dict(), p_join(ckpt_save_folder, f"model_{eph}_best.ckpt"))
                print(f"Epoch: {eph}/{epochs}, \t total score test: {score}, [best score: {best_score}]")
            print()

In [28]:
def create_dataloaders_reg(CONFIG, X, Z, Y):
    from sklearn.model_selection import train_test_split

    X_train, X_test, Z_train, Z_test, Y_train, Y_test = train_test_split(
        X, Z, Y, test_size=0.2, random_state=42)


    train_dataset = TensorDataset(torch.Tensor(X_train), torch.Tensor(Z_train), torch.Tensor(Y_train))
    test_dataset = TensorDataset(torch.Tensor(X_test), torch.Tensor(Z_test), torch.Tensor(Y_test))

    train_dataloader = DataLoader(train_dataset,
                                  shuffle=True,
                                  num_workers=CONFIG['cpu_workers'],
                                  batch_size=CONFIG['train_batch_size'],
                                  drop_last=True)

    test_dataloader = DataLoader(test_dataset,
                                 shuffle=False,
                                 num_workers=CONFIG['cpu_workers'],
                                 batch_size=CONFIG['test_batch_size'],
                                 drop_last=False)

    return train_dataloader, test_dataloader

In [29]:
def reg_create_dataset(folders, verbose=False, feature_nums=16):
    X_merged, Y_merged_1, Y_merged_2 = [], [], []
    for folder_path in tqdm(folders):
        X_path = p_join(folder_path, '2nd_exp_Input.txt')
        Y_path_down = p_join(folder_path, '2nd_exp_Topology_down.txt')
        Y_path_up = p_join(folder_path, '2nd_exp_Topology_up.txt')
        params_path = p_join(folder_path, '2nd_exp_Parameters.txt')

        X = utils.select_n_center_features(pd.read_csv(X_path).values, feature_nums)
        Y1 = pd.read_csv(params_path).values[:,:1]
        Y_down = pd.read_csv(Y_path_down).values
        Y_up = pd.read_csv(Y_path_up).values
        Y2 = np.array([utils.map_classes(Y_up[i], Y_down[i]) for i in range(Y_up.shape[0])])

        X_merged.append(X)
        Y_merged_1.append(Y1)
        Y_merged_2.append(Y2)

    # Merge and shuffle it!
    X, Y1, Y2 = np.concatenate(X_merged, axis=0), np.concatenate(Y_merged_1, axis=0), np.concatenate(Y_merged_2, axis=0)
    indexes_for_shuffle = np.random.permutation(np.arange(X.shape[0]))
    X = X[indexes_for_shuffle]
    Y1 = Y1[indexes_for_shuffle]
    Y2 = Y2[indexes_for_shuffle]
    if verbose:
        print('Dataset cteated!')
    return X, Y1, Y2

In [30]:
X, EXT_FEATS, Y = reg_create_dataset(FOLDERS)
X = utils.normalize_data(X)
EXT_FEATS = utils.normalize_data(EXT_FEATS)

100%|███████████████████████████████████████████| 16/16 [00:00<00:00, 17.30it/s]


In [31]:
CONFIG = {'ephs': 100,
          'train_batch_size': 1024,
          'test_batch_size': 64,
          'cpu_workers': 2,
          'save_ckpts': False,
          'criterion': lambda x, z, y: F.cross_entropy(x[0], y) + F.mse_loss(x[1], z)}

In [32]:
net = Mlp(X.shape[1], EXT_FEATS.shape[1])
utils.count_params(net)
train_dataloader, test_dataloader = create_dataloaders_reg(CONFIG, X, EXT_FEATS, Y)
optimizer = torch.optim.Adam(net.parameters(), lr=CONFIG.get('lr', 5e-3))
run_training(net, optimizer, CONFIG, train_dataloader, test_dataloader)

[INFO]: Model "Mlp" has 152517 trainable parameters


100%|███████████████████████████████████████████| 49/49 [00:02<00:00, 19.32it/s]


Epoch: 1/100, 	 total train loss: 2.109780637585387


100%|████████████████████████████████████████| 200/200 [00:00<00:00, 228.52it/s]


Epoch: 1/100, 	 total score test: 46.84691724622959, [best score: 46.84691724622959]



...

Epoch: 99/100, 	 total score test: 81.0971321403454, [best score: 81.33937641634758]



100%|███████████████████████████████████████████| 49/49 [00:05<00:00,  8.41it/s]


Epoch: 100/100, 	 total train loss: 0.48074000526447686


100%|█████████████████████████████████████████| 200/200 [00:02<00:00, 87.33it/s]

Epoch: 100/100, 	 total score test: 81.05806048292568, [best score: 81.33937641634758]



In [33]:
### Try more features

In [34]:
def reg_create_dataset(folders, verbose=False, feature_nums=16):
    X_merged, Y_merged_1, Y_merged_2 = [], [], []
    for folder_path in tqdm(folders):
        X_path = p_join(folder_path, '2nd_exp_Input.txt')
        Y_path_down = p_join(folder_path, '2nd_exp_Topology_down.txt')
        Y_path_up = p_join(folder_path, '2nd_exp_Topology_up.txt')
        params_path = p_join(folder_path, '2nd_exp_Parameters.txt')

        X = utils.select_n_center_features(pd.read_csv(X_path).values, feature_nums)
        Y1 = pd.read_csv(params_path).values[:,:6]
        Y_down = pd.read_csv(Y_path_down).values
        Y_up = pd.read_csv(Y_path_up).values
        Y2 = np.array([utils.map_classes(Y_up[i], Y_down[i]) for i in range(Y_up.shape[0])])

        X_merged.append(X)
        Y_merged_1.append(Y1)
        Y_merged_2.append(Y2)

    # Merge and shuffle it!
    X, Y1, Y2 = np.concatenate(X_merged, axis=0), np.concatenate(Y_merged_1, axis=0), np.concatenate(Y_merged_2, axis=0)
    indexes_for_shuffle = np.random.permutation(np.arange(X.shape[0]))
    X = X[indexes_for_shuffle]
    Y1 = Y1[indexes_for_shuffle]
    Y2 = Y2[indexes_for_shuffle]
    if verbose:
        print('Dataset cteated!')
    return X, Y1, Y2

In [35]:
X, EXT_FEATS, Y = reg_create_dataset(FOLDERS)
X = utils.normalize_data(X)
EXT_FEATS = utils.normalize_data(EXT_FEATS)

100%|███████████████████████████████████████████| 16/16 [00:01<00:00, 11.90it/s]


In [36]:
CONFIG = {'ephs': 100,
          'train_batch_size': 1024,
          'test_batch_size': 64,
          'cpu_workers': 2,
          'save_ckpts': False,
          'criterion': lambda x, z, y: F.cross_entropy(x[0], y) + F.mse_loss(x[1], z)}

In [37]:
net = Mlp(X.shape[1], EXT_FEATS.shape[1])
utils.count_params(net)
train_dataloader, test_dataloader = create_dataloaders_reg(CONFIG, X, EXT_FEATS, Y)
optimizer = torch.optim.Adam(net.parameters(), lr=CONFIG.get('lr', 5e-3))
run_training(net, optimizer, CONFIG, train_dataloader, test_dataloader)

[INFO]: Model "Mlp" has 152842 trainable parameters


100%|███████████████████████████████████████████| 49/49 [00:06<00:00,  7.50it/s]


Epoch: 1/100, 	 total train loss: 1.9466618664410649


100%|█████████████████████████████████████████| 200/200 [00:02<00:00, 71.90it/s]


Epoch: 1/100, 	 total score test: 50.37118074548722, [best score: 50.37118074548722]



...

Epoch: 99/100, 	 total score test: 81.12838946628116, [best score: 81.91763694615926]



100%|███████████████████████████████████████████| 49/49 [00:03<00:00, 14.70it/s]


Epoch: 100/100, 	 total train loss: 0.6045132315888697


100%|████████████████████████████████████████| 200/200 [00:01<00:00, 155.44it/s]

Epoch: 100/100, 	 total score test: 80.7611158865359, [best score: 81.91763694615926]



In [39]:
### Try more coef for reg
CONFIG = {'ephs': 100,
          'train_batch_size': 1024,
          'test_batch_size': 64,
          'cpu_workers': 2,
          'save_ckpts': False,
          'criterion': lambda x, z, y: F.cross_entropy(x[0], y) + 5*F.mse_loss(x[1], z)}

net = Mlp(X.shape[1], EXT_FEATS.shape[1])
utils.count_params(net)
train_dataloader, test_dataloader = create_dataloaders_reg(CONFIG, X, EXT_FEATS, Y)
optimizer = torch.optim.Adam(net.parameters(), lr=CONFIG.get('lr', 5e-3))
run_training(net, optimizer, CONFIG, train_dataloader, test_dataloader)

[INFO]: Model "Mlp" has 152842 trainable parameters


100%|███████████████████████████████████████████| 49/49 [00:03<00:00, 12.90it/s]


Epoch: 1/100, 	 total train loss: 5.398499527756049


100%|████████████████████████████████████████| 200/200 [00:01<00:00, 135.21it/s]


Epoch: 1/100, 	 total score test: 45.291865280925215, [best score: 45.291865280925215]



...

Epoch: 99/100, 	 total score test: 81.38626240525123, [best score: 81.62850668125341]



100%|███████████████████████████████████████████| 49/49 [00:02<00:00, 18.20it/s]


Epoch: 100/100, 	 total train loss: 1.9184109994343348


100%|████████████████████████████████████████| 200/200 [00:00<00:00, 222.64it/s]

Epoch: 100/100, 	 total score test: 81.30811909041182, [best score: 81.62850668125341]

